## Inference Demo
**Step 1.** Set these variables

In [1]:
CKPT_PATH = './lightning_logs/roberta-base/version_0/checkpoints/epoch=3-step=264.ckpt'
DATA_PATH = './data/en_gum'

**Step 2.** Load the imports

In [2]:
from datasets import load_from_disk
from functools import partial
from models.pos_classifier import POSClassifier
from transformers import AutoTokenizer, logging

import torch

logging.set_verbosity_error()

**Step 3.** Create these objects

In [3]:
ud = load_from_disk('./data/en_gum')
POS_map = ud['train'].info.features['upos'].feature.names

model = POSClassifier.load_from_checkpoint(CKPT_PATH)
tokenizer = AutoTokenizer.from_pretrained(model.hparams.encoder_name, add_prefix_space=True)
tokenize_fn = partial(tokenizer, return_tensors='pt')

**Step 4.** Define the inference method

In [4]:
@torch.no_grad()
def infer(sentence: str) -> list:
    e = tokenize_fn(sentence)
    o = model(e).squeeze(dim=0)

    return [POS_map[i] for i in o.argmax(dim=1)]

**Step 5.** Perform inference!

In [5]:
infer("Hello guys, I am a dog")

['INTJ', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'DET', 'NOUN']

**Step 6.** Profit?